### pseudocode for text processing problem
1. Create a list of all sentences.
- Scatter sentences among all the engines
- Call whatever text processing stuff is needed on each engine (co-occurrence dictionary)
- Call gather to get all the resulting dictionaries.

In [11]:
import numpy as np
from ipyparallel import Client
client = Client()
dview = client[:]

In [2]:
from nltk.corpus import state_union
from os import walk

raw_speeches = []
for (dirpath, dirname, filename) in walk("/home/tanner/nltk_data/corpora/state_union/"):
    for f in filename:
        raw_speeches.append(state_union.raw(f).lower())

In [3]:
%%px
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

stop_words = set(stopwords.words('english'))
def co_dict_sent(sentence,n,d):
    #words = sentence.strip().split(' ')
    #words = word_tokenize(sentence.lower())
    words = tokenizer.tokenize(sentence.lower())
    filtered = [w for w in words if w not in stop_words]
    for i in xrange(len(filtered)):
        for j in xrange(-n,n+1):
            if j != 0 and i+j >=0 and i+j <= len(filtered):
                try:
                    if filtered[i] not in d.keys():
                        d[filtered[i]] = [filtered[i+j]]
                    else:
                        d[filtered[i]].append(filtered[i+j])
                except Exception:
                    pass

In [4]:
%%px
d = dict()
for i, speech in enumerate(speeches):
    co_dict_sent(speech,5,d)

In [5]:
dictionaries = dview.gather("d").get()

In [6]:
combined_dict = dict()
for d in dictionaries:
    for key in d.keys():
        combined_dict[key] = []

In [7]:
for d in dictionaries:
    for key in d.keys():
        combined_dict[key] += d[key]

In [8]:
import scipy.sparse as spar

In [9]:
index = {word:i for i,word in enumerate(combined_dict.keys())}

In [12]:
rows = []
cols = []
for key, value in combined_dict.iteritems():
    for w in value:
        rows.append(index[key])
        cols.append(index[w])
data = np.ones(len(rows))

In [13]:
cooccurence = spar.coo_matrix((data, (rows, cols)), shape=(len(combined_dict), len(combined_dict))).tocsr()

In [24]:
dense = cooccurence.astype(np.int32).toarray()

In [27]:
import matplotlib.pyplot as plt
plt.spy(dense, markersize=1)
plt.show()